In [3]:
import mysql.connector

In [4]:
mydb = mysql.connector.connect(
    host='localhost',
    user='root',
    passwd='slow675',
    database='amd_project'
    )

In [5]:
print(mydb)

In [6]:
mycursor = mydb.cursor()

In [35]:
%run ../ddc/load_dataset.ipynb

In [36]:
subj_range = np.hstack((np.arange(2001,2002),np.arange(3001,3006)))

all_patients = [str(i) for i in subj_range]

In [37]:
X_all_p, y_all_p, subj_all_p, ts_all_p, hr_all_p = load_all_data(all_patients)

Loading 2001's data
Loading 3001's data
Loading 3002's data
Loading 3003's data
Loading 3004's data
Loading 3005's data
Finished loading


In [39]:
X_all_p.shape, ts_all_p.shape, hr_all_p.shape

((10998, 3), (10998,), (10998,))

In [47]:
X_dict = {
    'id': subj_all_p,
    'timestamp': ts_all_p,
    'x': [X_i[0] for X_i in X_all_p],
    'y': [X_i[1] for X_i in X_all_p],
    'z': [X_i[2] for X_i in X_all_p],
    'HR': hr_all_p,
    'label': y_all_p
}

df_all_p = pd.DataFrame(X_dict)

In [93]:
df_all_p['Milliseconds'] = df_all_p['timestamp'].apply(lambda x: x.split(' ')[1].split('.')[1])

In [99]:
df_all_p

,id,timestamp,x,y,z,HR,label,Milliseconds
0,2001,2019-01-16 13:59:26.666,-0.363528,-0.920057,0.197522,63.820263,0,666
1,2001,2019-01-16 13:59:26.827,-0.396265,-0.883412,0.212180,63.820263,0,827
2,2001,2019-01-16 13:59:26.987,-0.404571,-0.818426,0.193124,63.820263,0,987
3,2001,2019-01-16 13:59:27.147,-0.396753,-0.926409,0.155501,63.820263,0,147
4,2001,2019-01-16 13:59:27.309,-0.393822,-0.892207,0.199476,63.820263,0,309
5,2001,2019-01-16 13:59:27.469,-0.372323,-0.894161,0.208760,63.820263,0,469
6,2001,2019-01-16 13:59:27.631,-0.403594,-0.872173,0.220486,63.977630,0,631
7,2001,2019-01-16 13:59:27.79,-0.399196,-0.891229,0.210714,63.977630,0,79
8,2001,2019-01-16 13:59:27.95,-0.411412,-0.879991,0.230259,63.977630,0,95
9,2001,2019-01-16 13:59:28.111,-0.415321,-0.872173,0.228793,63.977630,0,111


In [97]:
sql = "INSERT INTO Patient (ID, Dateandtime, Milliseconds, X, Y, Z, HR, Label) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"

for row in zip(df_all_p['id'], 
               df_all_p['timestamp'], 
               df_all_p['Milliseconds'],
               df_all_p['x'], 
               df_all_p['y'], 
               df_all_p['z'], 
               df_all_p['HR'], 
               [None for i in range(len(df_all_p))]):
    
    mycursor.execute(sql, row)

mydb.commit()

('2001', '2019-01-16 13:59:26.666', '666', -0.36352777551020404, -0.9200574489795917, 0.19752164285714283, 63.820263, None)
('2001', '2019-01-16 13:59:26.827', '827', -0.3962648163265306, -0.883411581632653, 0.21217999999999998, 63.820263, None)
('2001', '2019-01-16 13:59:26.987', '987', -0.40457122448979593, -0.8184261224489795, 0.19312412244897958, 63.820263, None)
('2001', '2019-01-16 13:59:27.147', '147', -0.3967534285714286, -0.9264094897959183, 0.15550094897959185, 63.820263, None)
('2001', '2019-01-16 13:59:27.309', '309', -0.39382175510204076, -0.8922066326530611, 0.19947609183673465, 63.820263, None)
('2001', '2019-01-16 13:59:27.469', '469', -0.3723227959183673, -0.8941610204081631, 0.20875972448979593, 63.820263, None)
('2001', '2019-01-16 13:59:27.631', '631', -0.40359399999999995, -0.872173469387755, 0.22048642857142856, 63.977630000000005, None)
('2001', '2019-01-16 13:59:27.79', '79', -0.3991964897959183, -0.891229387755102, 0.21071417346938773, 63.977630000000005, None)

('2001', '2019-01-16 14:01:56.365', '365', 0.7402480816326531, -0.6288444285714285, 0.23099160204081634, 64.50064, None)
('2001', '2019-01-16 14:01:56.526', '526', 0.7402480816326531, -0.6239582653061224, 0.23441187755102036, 64.50064, None)
('2001', '2019-01-16 14:01:56.686', '686', 0.7441569795918367, -0.6312874897959183, 0.23245742857142854, 64.50064, None)
('2001', '2019-01-16 14:01:56.847', '847', 0.741225306122449, -0.6205379591836735, 0.2319688163265306, 64.50064, None)
('2001', '2019-01-16 14:01:57.007', '007', 0.7451341836734693, -0.6273785714285713, 0.2319688163265306, 64.84744, None)
('2001', '2019-01-16 14:01:57.168', '168', 0.7407366938775509, -0.62689, 0.2353891020408163, 64.84744, None)
('2001', '2019-01-16 14:01:57.329', '329', 0.744645612244898, -0.620049357142857, 0.23734355102040813, 64.84744, None)
('2001', '2019-01-16 14:01:57.491', '491', 0.7397594897959182, -0.6264013673469387, 0.237832193877551, 64.84744, None)
('2001', '2019-01-16 14:01:57.65', '65', 0.74513418

('2001', '2019-01-16 14:00:22.242', '242', -0.9240884693877551, -0.37488801020408163, 0.024552783673469386, 63.57512, None)
('2001', '2019-01-16 14:00:22.403', '403', -0.9162707142857142, -0.37879690816326533, 0.01868943163265306, 63.57512, None)
('2001', '2019-01-16 14:00:22.564', '564', -0.918713775510204, -0.36560438775510207, 0.024552783673469386, 63.57512, None)
('2001', '2019-01-16 14:00:22.724', '724', -0.9289746938775509, -0.3773310714285714, 0.03725671020408163, 64.32609599999999, None)
('2001', '2019-01-16 14:00:22.885', '885', -0.9201795918367347, -0.38221719387755104, 0.02504139489795918, 64.32609599999999, None)
('2001', '2019-01-16 14:00:23.046', '046', -0.9192023979591837, -0.3719563265306122, 0.024552783673469386, 64.32609599999999, None)
('2001', '2019-01-16 14:00:23.206', '206', -0.9231113265306121, -0.3719563265306122, 0.03383642244897959, 64.32609599999999, None)
('2001', '2019-01-16 14:00:23.367', '367', -0.9250657142857143, -0.3758652346938775, 0.02601861938775510

('2001', '2019-01-16 14:05:19.75', '75', -1.1565460204081632, -0.2835174489795918, 0.02394201632653061, 84.67343000000001, None)
('2001', '2019-01-16 14:05:19.909', '909', -0.7338961224489796, -0.2502917959183673, 0.05130432040816326, 84.67343000000001, None)
('2001', '2019-01-16 14:05:20.071', '071', -0.8653329591836735, -0.28205161224489794, 0.07817801020408163, 84.67343000000001, None)
('2001', '2019-01-16 14:05:20.231', '231', -0.815494387755102, -0.2683704693877551, 0.10798338775510204, 84.67343000000001, None)
('2001', '2019-01-16 14:05:20.391', '391', -1.4414071428571429, -0.31478867346938777, 0.04348652040816326, 84.67343000000001, None)
('2001', '2019-01-16 14:05:20.552', '552', -0.9469311224489796, -0.2908466530612245, 0.0004886125918367347, 84.87380999999999, None)
('2001', '2019-01-16 14:05:20.712', '712', -0.8648442857142856, -0.2527348571428571, -0.0028095223469387755, 84.87380999999999, None)
('2001', '2019-01-16 14:05:20.873', '873', -0.9869974489795917, -0.313811448979

('2001', '2019-01-16 14:08:08.296', '296', -0.5677678163265305, -0.49655253061224486, 0.6351963877551019, 85.46423, None)
('2001', '2019-01-16 14:08:08.457', '457', -0.7153288163265306, -0.5972067346938775, 0.6610928265306123, 86.94528000000001, None)
('2001', '2019-01-16 14:08:08.619', '619', -0.3982192653061224, -0.9343494897959183, 0.6928526530612245, 86.94528000000001, None)
('2001', '2019-01-16 14:08:08.779', '779', -0.41190040816326523, -0.5649583163265306, 0.4930101020408163, 86.94528000000001, None)
('2001', '2019-01-16 14:08:08.941', '941', -0.6259127551020407, -0.6646353061224489, 0.2462607551020408, 86.94528000000001, None)
('2001', '2019-01-16 14:08:09.102', '102', -0.2252504081632653, -0.6988381326530613, 0.6449686224489796, 86.94528000000001, None)
('2001', '2019-01-16 14:08:09.26', '26', -0.1695485714285714, -0.7056787755102041, 0.6728195204081632, 86.94528000000001, None)
('2001', '2019-01-16 14:08:09.423', '423', -0.040066232653061216, -0.7120307142857142, 0.7143515918

...

('3004', '2019-03-05 12:52:53.309', '309', 0.875593775510204, 0.37769753061224487, 0.2585982040816326, 111.27967, None)
('3004', '2019-03-05 12:52:53.469', '469', 0.8736393877551021, 0.3864925510204081, 0.2644615714285714, 111.32576, None)
('3004', '2019-03-05 12:52:53.629', '629', 0.8760824489795919, 0.3884470204081632, 0.27618826530612245, 111.32576, None)
('3004', '2019-03-05 12:52:53.788', '788', 0.8692418367346939, 0.37476585714285704, 0.2752110408163265, 111.32576, None)
('3004', '2019-03-05 12:52:53.95', '95', 0.9010016326530612, 0.37036834693877546, 0.2781427142857143, 111.32576, None)
('3004', '2019-03-05 12:52:54.109', '109', 0.9058877551020408, 0.28095224489795917, 0.27276797959183674, 111.32576, None)
('3004', '2019-03-05 12:52:54.272', '272', 0.9503514285714285, 0.24870380612244897, 0.21217999999999998, 111.32576, None)
('3004', '2019-03-05 12:52:54.432', '432', 0.9371588775510205, 0.30049673469387755, 0.016246367346938774, 111.32576, None)
('3004', '2019-03-05 12:52:54.59

('3005', '2019-03-05 13:00:24.334', '334', 0.5405276530612244, 0.6507098469387755, 0.5319769387755101, 63.859306000000004, None)
('3005', '2019-03-05 13:00:24.492', '492', 0.5390618367346939, 0.651198469387755, 0.5432150306122447, 62.725131999999995, None)
('3005', '2019-03-05 13:00:24.653', '653', 0.5400390510204082, 0.6497325510204081, 0.5373516836734693, 62.725131999999995, None)
('3005', '2019-03-05 13:00:24.813', '813', 0.5419934999999999, 0.6492439795918367, 0.5363744591836734, 62.725131999999995, None)
('3005', '2019-03-05 13:00:24.975', '975', 0.5419934999999999, 0.6492439795918367, 0.5309997244897958, 62.725131999999995, None)
('3005', '2019-03-05 13:00:25.134', '134', 0.5395504387755102, 0.6482667346938775, 0.5353972448979591, 62.725131999999995, None)
('3005', '2019-03-05 13:00:25.295', '295', 0.5429707142857142, 0.6492439795918367, 0.5393061224489796, 62.725131999999995, None)
('3005', '2019-03-05 13:00:25.456', '456', 0.541016275510204, 0.6477781224489795, 0.53393139795918

('3005', '2019-03-05 13:03:26.787', '787', -0.03579087142857142, 0.8135399489795917, -0.5790059183673469, 70.36943000000001, None)
('3005', '2019-03-05 13:03:26.949', '949', -0.03530225918367346, 0.8140285714285713, -0.5785173469387754, 70.36943000000001, None)
('3005', '2019-03-05 13:03:27.108', '108', -0.036279485714285714, 0.8140285714285713, -0.5785173469387754, 70.36943000000001, None)
('3005', '2019-03-05 13:03:27.271', '271', -0.03432503469387754, 0.8145171734693877, -0.5775401020408163, 70.36943000000001, None)
('3005', '2019-03-05 13:03:27.432', '432', -0.03579087142857142, 0.8140285714285713, -0.5760742857142856, 70.36943000000001, None)
('3005', '2019-03-05 13:03:27.59', '59', -0.03432503469387754, 0.8150057857142856, -0.5770514897959182, 70.36943000000001, None)
('3005', '2019-03-05 13:03:27.753', '753', -0.03530225918367346, 0.8145171734693877, -0.5760742857142856, 70.36943000000001, None)
('3005', '2019-03-05 13:03:27.911', '911', -0.03676809795918367, 0.815494387755102, 

('3005', '2019-03-05 13:01:11.391', '391', 0.9519394897959184, 0.3135671428571429, 0.03603517959183673, 80.776924, None)
('3005', '2019-03-05 13:01:11.554', '554', 0.9465647959183674, 0.3121012857142857, 0.03847824081632653, 80.776924, None)
('3005', '2019-03-05 13:01:11.712', '712', 0.9407014285714287, 0.309169612244898, 0.04873910714285714, 85.30093000000001, None)
('3005', '2019-03-05 13:01:11.873', '873', 0.9382583673469389, 0.308681, 0.04434159183673469, 85.30093000000001, None)
('3005', '2019-03-05 13:01:12.034', '034', 0.9436330612244896, 0.3062379387755102, 0.04776187959183673, 85.30093000000001, None)
('3005', '2019-03-05 13:01:12.194', '194', 0.9426559183673469, 0.3204077142857143, 0.04825049183673469, 85.30093000000001, None)
('3005', '2019-03-05 13:01:12.356', '356', 0.9494964285714284, 0.3135671428571429, 0.04825049183673469, 85.30093000000001, None)
('3005', '2019-03-05 13:01:12.516', '516', 0.9323950000000001, 0.32138493877551016, 0.049227719387755095, 85.30093000000001,

('3005', '2019-03-05 13:05:41.526', '526', 0.914316275510204, 0.3284698163265306, 0.3012296632653061, 71.407646, None)
('3005', '2019-03-05 13:05:41.688', '688', 0.9226226530612245, 0.17651129591836734, 0.18933737755102042, 71.407646, None)
('3005', '2019-03-05 13:05:41.849', '849', 0.9846765306122448, 0.2942669387755102, 0.0725589693877551, 71.407646, None)
('3005', '2019-03-05 13:05:42.009', '009', 1.1038979591836733, 0.2771654897959183, 0.028583836734693873, 79.89375, None)
('3005', '2019-03-05 13:05:42.168', '168', 0.8371155612244896, 0.2038736020408163, 0.0608322693877551, 79.89375, None)
('3005', '2019-03-05 13:05:42.33', '33', 0.9607344897959184, 0.41642010204081625, 0.058877816326530615, 79.89375, None)
('3005', '2019-03-05 13:05:42.49', '49', 0.9382583673469389, 0.3675588265306122, 0.07353619387755102, 79.89375, None)
('3005', '2019-03-05 13:05:42.649', '649', 1.1488504081632653, 0.3216292448979592, 0.08330844489795917, 79.89375, None)
('3005', '2019-03-05 13:05:42.811', '811'

In [ ]:
sql = "INSERT INTO AllDaySummary (ID, DateTimeFrom, DateTimeUntil, DurationSit, DurationSleep, DurationStand, DurationWalk)\
        VALUES (%s, %s, %s, %s, %s, %s, %s)"

for

In [100]:
mycursor.execute("SELECT * FROM Patient")

myresult = mycursor.fetchall()

print(len(myresult))

for x in myresult:
    print(x)
    

10998
(2001, datetime.datetime(2019, 1, 16, 13, 58, 48), '117', -0.9445, -0.2977, 0.0861, 68.2053, None)
(2001, datetime.datetime(2019, 1, 16, 13, 58, 48), '278', -0.945, -0.2962, 0.0807, 68.2053, None)
(2001, datetime.datetime(2019, 1, 16, 13, 58, 48), '439', -0.9455, -0.2977, 0.0822, 67.3655, None)
(2001, datetime.datetime(2019, 1, 16, 13, 58, 48), '956', -0.9455, -0.2938, 0.0856, 68.2053, None)
(2001, datetime.datetime(2019, 1, 16, 13, 58, 49), '08', -0.9474, -0.3075, 0.0837, 67.3655, None)
(2001, datetime.datetime(2019, 1, 16, 13, 58, 49), '243', -0.9455, -0.2948, 0.0768, 67.3655, None)
(2001, datetime.datetime(2019, 1, 16, 13, 58, 49), '402', -0.945, -0.3026, 0.0739, 67.3655, None)
(2001, datetime.datetime(2019, 1, 16, 13, 58, 49), '6', -0.9435, -0.3011, 0.0842, 67.3655, None)
(2001, datetime.datetime(2019, 1, 16, 13, 58, 49), '76', -0.943, -0.2957, 0.0778, 67.3655, None)
(2001, datetime.datetime(2019, 1, 16, 13, 58, 49), '922', -0.9425, -0.2884, 0.0807, 67.3655, None)
(2001, date

(2001, datetime.datetime(2019, 1, 16, 14, 4, 48), '779', -0.8477, -0.2092, 0.0034, 81.9518, None)
(2001, datetime.datetime(2019, 1, 16, 14, 4, 48), '941', -0.9313, -0.2435, 0.002, 81.9518, None)
(2001, datetime.datetime(2019, 1, 16, 14, 4, 49), '065', -0.8678, -0.2102, 0.0059, 80.9688, None)
(2001, datetime.datetime(2019, 1, 16, 14, 4, 49), '226', -1.0051, -0.265, 0.0166, 80.4773, None)
(2001, datetime.datetime(2019, 1, 16, 14, 4, 49), '386', -0.8976, -0.3192, 0.1222, 80.4773, None)
(2001, datetime.datetime(2019, 1, 16, 14, 4, 49), '583', -0.9293, -0.3358, 0.1593, 80.9688, None)
(2001, datetime.datetime(2019, 1, 16, 14, 4, 49), '743', -1.203, -0.3715, 0.0806, 80.9688, None)
(2001, datetime.datetime(2019, 1, 16, 14, 4, 49), '904', -0.9474, -0.3016, 0.0269, 80.9688, None)
(2001, datetime.datetime(2019, 1, 16, 14, 4, 50), '03', -0.9127, -0.3461, 0.0845, 80.4773, None)
(2001, datetime.datetime(2019, 1, 16, 14, 4, 50), '19', -1.0168, -0.2889, 0.0332, 80.4773, None)
(2001, datetime.datetime(

(2001, datetime.datetime(2019, 1, 16, 14, 9, 34), '969', -0.1322, -1.1689, 0.345, 91.8844, None)
(2001, datetime.datetime(2019, 1, 16, 14, 9, 35), '093', -0.9843, -0.672, 0.1896, 92.8586, None)
(2001, datetime.datetime(2019, 1, 16, 14, 9, 35), '255', -0.8866, -0.6045, 0.0748, 92.8586, None)
(2001, datetime.datetime(2019, 1, 16, 14, 9, 35), '417', -0.8148, -0.5058, 0.1051, 92.8586, None)
(2001, datetime.datetime(2019, 1, 16, 14, 9, 35), '612', -0.2587, -1.0179, 0.1226, 91.8844, None)
(2001, datetime.datetime(2019, 1, 16, 14, 9, 35), '775', -0.3994, -1.0409, 0.0518, 92.8586, None)
(2001, datetime.datetime(2019, 1, 16, 14, 9, 35), '934', -0.5436, -0.6749, 0.0777, 92.8586, None)
(2001, datetime.datetime(2019, 1, 16, 14, 9, 36), '058', -0.868, -0.5117, 0.1568, 93.3457, None)
(2001, datetime.datetime(2019, 1, 16, 14, 9, 36), '219', -0.7224, -0.519, 0.1862, 93.3457, None)
(2001, datetime.datetime(2019, 1, 16, 14, 9, 36), '38', -0.824, -0.5454, 0.2399, 93.3457, None)
(2001, datetime.datetime(2

(3001, datetime.datetime(2019, 3, 5, 10, 6, 55), '556', -0.6673, -0.6616, 0.3623, 90.7017, None)
(3001, datetime.datetime(2019, 3, 5, 10, 6, 55), '717', -0.6644, -0.6611, 0.3633, 90.7017, None)
(3001, datetime.datetime(2019, 3, 5, 10, 6, 55), '878', -0.6678, -0.6601, 0.3633, 90.7017, None)
(3001, datetime.datetime(2019, 3, 5, 10, 6, 56), '002', -0.6688, -0.6572, 0.3613, 90.9527, None)
(3001, datetime.datetime(2019, 3, 5, 10, 6, 56), '161', -0.6727, -0.6591, 0.3608, 90.9527, None)
(3001, datetime.datetime(2019, 3, 5, 10, 6, 56), '324', -0.6712, -0.6591, 0.3647, 90.9527, None)
(3001, datetime.datetime(2019, 3, 5, 10, 6, 56), '483', -0.6737, -0.6591, 0.3623, 90.9527, None)
(3001, datetime.datetime(2019, 3, 5, 10, 6, 56), '52', -0.6698, -0.6591, 0.3623, 90.9527, None)
(3001, datetime.datetime(2019, 3, 5, 10, 6, 56), '679', -0.6639, -0.6621, 0.3647, 90.9527, None)
(3001, datetime.datetime(2019, 3, 5, 10, 6, 56), '841', -0.6668, -0.6582, 0.3662, 90.9527, None)
(3001, datetime.datetime(2019, 

(3002, datetime.datetime(2019, 3, 5, 12, 24, 1), '037', 0.9631, 0.2588, 0.0999, 63.6348, None)
(3002, datetime.datetime(2019, 3, 5, 12, 24, 1), '196', 0.964, 0.2598, 0.118, 63.6348, None)
(3002, datetime.datetime(2019, 3, 5, 12, 24, 1), '358', 0.9601, 0.2593, 0.1077, 63.4215, None)
(3002, datetime.datetime(2019, 3, 5, 12, 24, 1), '554', 0.9606, 0.2613, 0.097, 63.6348, None)
(3002, datetime.datetime(2019, 3, 5, 12, 24, 1), '715', 0.9606, 0.2554, 0.1102, 63.6348, None)
(3002, datetime.datetime(2019, 3, 5, 12, 24, 1), '874', 0.9611, 0.2613, 0.1068, 63.6348, None)
(3002, datetime.datetime(2019, 3, 5, 12, 24, 2), '0', 0.9699, 0.2642, 0.1058, 63.4215, None)
(3002, datetime.datetime(2019, 3, 5, 12, 24, 2), '16', 0.9557, 0.2608, 0.1107, 63.4215, None)
(3002, datetime.datetime(2019, 3, 5, 12, 24, 2), '319', 0.9601, 0.2657, 0.1033, 63.4215, None)
(3002, datetime.datetime(2019, 3, 5, 12, 24, 2), '482', 0.9635, 0.2632, 0.1151, 63.2624, None)
(3002, datetime.datetime(2019, 3, 5, 12, 24, 2), '518', 

...

(3003, datetime.datetime(2019, 3, 5, 12, 35, 28), '383', -0.9179, -0.304, 0.0114, 93.4225, None)
(3003, datetime.datetime(2019, 3, 5, 12, 35, 28), '582', -0.91, -0.39, -0.0286, 94.3588, None)
(3003, datetime.datetime(2019, 3, 5, 12, 35, 28), '739', -0.8934, -0.3553, 0.0187, 94.3588, None)
(3003, datetime.datetime(2019, 3, 5, 12, 35, 28), '902', -0.9076, -0.35, 0.0206, 93.4225, None)
(3003, datetime.datetime(2019, 3, 5, 12, 35, 29), '026', -0.9164, -0.3187, -0.1092, 92.9543, None)
(3003, datetime.datetime(2019, 3, 5, 12, 35, 29), '187', -0.9413, -0.2786, -0.0618, 92.9543, None)
(3003, datetime.datetime(2019, 3, 5, 12, 35, 29), '347', -0.9653, -0.4379, -0.0149, 92.9543, None)
(3003, datetime.datetime(2019, 3, 5, 12, 35, 29), '545', -0.9354, -0.2303, 0.0348, 93.4225, None)
(3003, datetime.datetime(2019, 3, 5, 12, 35, 29), '704', -0.9052, -0.3285, 0.0114, 93.4225, None)
(3003, datetime.datetime(2019, 3, 5, 12, 35, 29), '864', -0.9169, -0.3905, -0.0183, 93.4225, None)
(3003, datetime.dateti

(3003, datetime.datetime(2019, 3, 5, 12, 39, 53), '911', -1.2218, -0.7492, 0.5543, 54.4802, None)
(3003, datetime.datetime(2019, 3, 5, 12, 39, 54), '038', -0.9012, -0.5381, 0.2871, 54.5352, None)
(3003, datetime.datetime(2019, 3, 5, 12, 39, 54), '196', -0.827, -0.585, 0.1893, 54.5352, None)
(3003, datetime.datetime(2019, 3, 5, 12, 39, 54), '359', -0.9892, -0.497, 0.1825, 54.5352, None)
(3003, datetime.datetime(2019, 3, 5, 12, 39, 54), '554', -1.0767, -0.2102, 0.245, 54.5352, None)
(3003, datetime.datetime(2019, 3, 5, 12, 39, 54), '715', -0.9276, -0.2405, 0.2519, 54.5352, None)
(3003, datetime.datetime(2019, 3, 5, 12, 39, 54), '876', -0.9628, -0.4492, 0.2333, 54.5352, None)
(3003, datetime.datetime(2019, 3, 5, 12, 39, 55), '0', -0.9301, -0.4814, 0.3105, 54.5627, None)
(3003, datetime.datetime(2019, 3, 5, 12, 39, 55), '162', -0.9374, -0.608, 0.2792, 54.5627, None)
(3003, datetime.datetime(2019, 3, 5, 12, 39, 55), '322', -1.0845, -0.3881, 0.1454, 54.5627, None)
(3003, datetime.datetime(20

(3004, datetime.datetime(2019, 3, 5, 12, 50, 15), '084', 0.3831, 0.9179, 0.0164, 127.1822, None)
(3004, datetime.datetime(2019, 3, 5, 12, 50, 15), '246', 0.3797, 0.9183, 0.0183, 127.1822, None)
(3004, datetime.datetime(2019, 3, 5, 12, 50, 15), '407', 0.3816, 0.9149, 0.0237, 127.1822, None)
(3004, datetime.datetime(2019, 3, 5, 12, 50, 15), '603', 0.3904, 0.9135, 0.0159, 128.4638, None)
(3004, datetime.datetime(2019, 3, 5, 12, 50, 15), '775', 0.3865, 0.913, 0.0173, 127.1822, None)
(3004, datetime.datetime(2019, 3, 5, 12, 50, 15), '924', 0.3831, 0.9144, 0.0149, 127.1822, None)
(3004, datetime.datetime(2019, 3, 5, 12, 50, 16), '048', 0.3816, 0.9154, 0.0154, 126.5414, None)
(3004, datetime.datetime(2019, 3, 5, 12, 50, 16), '21', 0.3836, 0.9159, 0.012, 126.5414, None)
(3004, datetime.datetime(2019, 3, 5, 12, 50, 16), '369', 0.3836, 0.9164, 0.0095, 126.5414, None)
(3004, datetime.datetime(2019, 3, 5, 12, 50, 16), '567', 0.3792, 0.9164, 0.0252, 127.1822, None)
(3004, datetime.datetime(2019, 3,

(3005, datetime.datetime(2019, 3, 5, 12, 59, 41), '965', 0.9637, 0.2261, 0.037, 65.7057, None)
(3005, datetime.datetime(2019, 3, 5, 12, 59, 42), '091', 0.9544, 0.2349, 0.0468, 65.5005, None)
(3005, datetime.datetime(2019, 3, 5, 12, 59, 42), '25', 0.9578, 0.2242, 0.0429, 65.5005, None)
(3005, datetime.datetime(2019, 3, 5, 12, 59, 42), '412', 0.9656, 0.2237, 0.0419, 65.5005, None)
(3005, datetime.datetime(2019, 3, 5, 12, 59, 42), '608', 0.9788, 0.2349, 0.0409, 65.5005, None)
(3005, datetime.datetime(2019, 3, 5, 12, 59, 42), '77', 0.9729, 0.23, 0.0458, 65.5005, None)
(3005, datetime.datetime(2019, 3, 5, 12, 59, 42), '929', 0.9568, 0.232, 0.0541, 65.5005, None)
(3005, datetime.datetime(2019, 3, 5, 12, 59, 43), '054', 0.9602, 0.2251, 0.0351, 64.6814, None)
(3005, datetime.datetime(2019, 3, 5, 12, 59, 43), '214', 0.9539, 0.2271, 0.0395, 64.6814, None)
(3005, datetime.datetime(2019, 3, 5, 12, 59, 43), '375', 0.9598, 0.2178, 0.0355, 64.6814, None)
(3005, datetime.datetime(2019, 3, 5, 12, 59, 4

(3005, datetime.datetime(2019, 3, 5, 13, 3, 43), '332', -0.0451, 0.8399, -0.5394, 63.2806, None)
(3005, datetime.datetime(2019, 3, 5, 13, 3, 43), '49', -0.0456, 0.8409, -0.5385, 63.265, None)
(3005, datetime.datetime(2019, 3, 5, 13, 3, 43), '528', -0.0456, 0.8389, -0.5409, 63.2806, None)
(3005, datetime.datetime(2019, 3, 5, 13, 3, 43), '687', -0.0441, 0.8385, -0.5419, 63.2806, None)
(3005, datetime.datetime(2019, 3, 5, 13, 3, 43), '849', -0.0451, 0.8394, -0.5404, 63.2806, None)
(3005, datetime.datetime(2019, 3, 5, 13, 3, 44), '134', -0.047, 0.8414, -0.5375, 63.265, None)
(3005, datetime.datetime(2019, 3, 5, 13, 3, 44), '293', -0.0446, 0.8414, -0.5385, 63.265, None)
(3005, datetime.datetime(2019, 3, 5, 13, 3, 44), '456', -0.0436, 0.8419, -0.5365, 63.265, None)
(3005, datetime.datetime(2019, 3, 5, 13, 3, 44), '653', -0.0456, 0.8419, -0.537, 63.265, None)
(3005, datetime.datetime(2019, 3, 5, 13, 3, 44), '812', -0.0451, 0.8424, -0.538, 63.265, None)
(3005, datetime.datetime(2019, 3, 5, 13,

In [82]:
sql = "DELETE FROM Patient"

mycursor.execute(sql)

mydb.commit()

In [ ]:
mycursor.execute("DROP TABLE AllDaySummary")
mycursor.execute("DROP TABLE ActivityAnalysis")
# mycursor.execute("DROP TABLE Patient")